#### Смешиваем файл голоса с шумом

In [4]:
import soundfile
import librosa 

from dsp import mixer

In [11]:
SAMPLE_RATE = 48000

test_signal, test_file_rate = soundfile.read('../data/test/gt.wav')
noise, noise_rate = soundfile.read('../data/461143__15gpanskahladikova_danuse__15_metro_line_c_departure.wav')

noise = noise[:test_signal.shape[0]][:, 1]

In [24]:
snr_values = [-5, 0, 5, 10]

for snr in snr_values:
    mixed_signal = mixer(test_signal, noise, snr)

    soundfile.write(f'../data/mixed/mixed_{snr}dB.wav', mixed_signal, SAMPLE_RATE)

#### Считаем метрики 

##### Аналитические метрики

In [30]:
from torchmetrics.functional.audio import signal_distortion_ratio, scale_invariant_signal_distortion_ratio
import torch
from pathlib import Path
import pandas as pd

In [39]:
path2mixed = Path('../data/mixed/')

files = []
sdrs = []
si_sdrs = []


for file in path2mixed.glob('*.wav'):
    mixed_signal, _ = soundfile.read(file)

    files.append(file.parts[-1])
    sdrs.append(signal_distortion_ratio(torch.Tensor(mixed_signal), torch.Tensor(test_signal)).item())
    si_sdrs.append(scale_invariant_signal_distortion_ratio(torch.Tensor(mixed_signal), torch.Tensor(test_signal)).item())

analytic_table = pd.DataFrame({
    'file' : files,
    'SDR' : sdrs,
    'SI-SDR' : si_sdrs
})

In [40]:
analytic_table

,file,SDR,SI-SDR
0,mixed_5dB.wav,2.975620,2.969908
1,mixed_-5dB.wav,-7.030280,-7.053289
2,mixed_0dB.wav,-2.028577,-2.038438
3,mixed_10dB.wav,7.978998,7.974598


##### NISQA

In [43]:
!python ../NISQA/run_predict.py --mode predict_dir --pretrained_model ../NISQA/weights/nisqa.tar --data_dir ../data/mixed/  --output_dir ../nisqa_out/

Device: cpu
Model architecture: NISQA_DIM
Loaded pretrained model from ../NISQA/weights/nisqa.tar
# files: 4
---> Predicting ...
           deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
 mixed_5dB.wav  1.465870  1.280670  4.336345  3.153044   2.823355 NISQAv2
mixed_-5dB.wav  0.627106  1.359880  2.740413  1.221595   1.424671 NISQAv2
 mixed_0dB.wav  0.863872  1.335018  3.543309  2.061064   1.896784 NISQAv2
mixed_10dB.wav  2.225692  1.343056  4.428701  3.637326   3.289922 NISQAv2


##### DNSMOS

In [45]:
%cd ../DNS-Challenge/DNSMOS/
!python dnsmos_local.py -t ../../data/mixed/ -o ../../dnsmos_out/DNSMOS_results.csv
%cd ../../notebooks

/home/alexey/dsp/DNS-Challenge/DNSMOS


/home/alexey/miniconda3/envs/dsp/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 2750.81it/s]
4it [00:01,  2.82it/s]
/home/alexey/dsp/notebooks


#### Собираем все в 1 таблицу

In [56]:
nisqa_table = pd.read_csv('../nisqa_out/NISQA_results.csv').iloc[:, :-1]
dnsmos_table = pd.read_csv('../dnsmos_out/DNSMOS_results.csv')[['filename', 'P808_MOS']]

dnsmos_table.filename = dnsmos_table.filename.apply(lambda x: x.split('/')[-1])

In [62]:
result = pd.merge(analytic_table, nisqa_table, left_on='file', right_on='deg')
result = pd.merge(result, dnsmos_table, left_on='file', right_on='filename')

result.drop(['filename', 'deg'], axis=1, inplace=True)

In [63]:
result

,file,SDR,SI-SDR,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,P808_MOS
0,mixed_5dB.wav,2.975620,2.969908,1.465870,1.280670,4.336345,3.153045,2.823355,2.493290
1,mixed_-5dB.wav,-7.030280,-7.053289,0.627106,1.359880,2.740413,1.221595,1.424671,2.290628
2,mixed_0dB.wav,-2.028577,-2.038438,0.863872,1.335018,3.543309,2.061063,1.896784,2.432672
3,mixed_10dB.wav,7.978998,7.974598,2.225692,1.343056,4.428701,3.637326,3.289922,2.863240


#### Выводы

1) Аналитичесие метрики качественно коррелируют с уровнем шума
2) Перцептульные мтерики тоже качественно коррелируют с SNR